In [201]:
from itertools import product

G = PermutationGroup([
    [(1,2,3,4),(5,6,7,8),(9,10,11,12),(13,14,15,16),(17,18,19,20),(21,22,23,24),(25,26,27,28),(29,30,31,32)],
    [(1,24,3,22),(2,23,4,21),(5,14,7,16),(6,13,8,15),(9,27,11,25),(10,26,12,28),(17,31,19,29),(18,30,20,32)],
    [(1,5,12,30),(2,6,9,31),(3,7,10,32),(4,8,11,29),(13,25,19,21),(14,26,20,22),(15,27,17,23),(16,28,18,24)],
    [(1,26),(2,27),(3,28),(4,25),(5,14),(6,15),(7,16),(8,13),(9,23),(10,24),(11,21),(12,22),(17,31),(18,32),(19,29),(20,30)]
])

# (C4 × (C4 ⋊ C2 ) ) ⋊17 C2
elements = [*G]
e_ord = {}
for o,e in ((e.order(), e) for e in elements):
    if o not in e_ord: e_ord[o] = []
    e_ord[o].append(e)

for a,b,c,d in product(e_ord[4], e_ord[4], e_ord[2], e_ord[2]):
    if a == b or c == d: continue
    if len(set(a^x * b^y * c^z * d^w for x,y,z,w in product(*map(range, (4,4,2,2))))) != 64: continue
    if b*c != c*b^(-1): continue
    if a*b*a^(-1) != b: continue
    if a*c*a^(-1) != c: continue
    break
a,b,c,d

((1,4,3,2)(5,8,7,6)(9,12,11,10)(13,16,15,14)(17,20,19,18)(21,24,23,22)(25,28,27,26)(29,32,31,30),
 (1,30,12,5)(2,31,9,6)(3,32,10,7)(4,29,11,8)(13,21,19,25)(14,22,20,26)(15,23,17,27)(16,24,18,28),
 (1,22)(2,23)(3,24)(4,21)(5,20)(6,17)(7,18)(8,19)(9,27)(10,28)(11,25)(12,26)(13,29)(14,30)(15,31)(16,32),
 (2,4)(5,32)(6,31)(7,30)(8,29)(9,11)(13,17)(14,20)(15,19)(16,18)(22,24)(26,28))

In [30]:
{a^x * b^y * c^z * d^w: (x,y,z,w) for x,y,z,w in product(*map(range, (4,4,2,2)))}

['()',
 '(2,4)(5,32)(6,31)(7,30)(8,29)(9,11)(13,17)(14,20)(15,19)(16,18)(22,24)(26,28)',
 '(1,22)(2,23)(3,24)(4,21)(5,20)(6,17)(7,18)(8,19)(9,27)(10,28)(11,25)(12,26)(13,29)(14,30)(15,31)(16,32)',
 '(1,24,3,22)(2,23,4,21)(5,14,7,16)(6,13,8,15)(9,27,11,25)(10,26,12,28)(17,31,19,29)(18,30,20,32)',
 '(1,30,12,5)(2,31,9,6)(3,32,10,7)(4,29,11,8)(13,21,19,25)(14,22,20,26)(15,23,17,27)(16,24,18,28)',
 '(1,7,3,5)(2,6,4,8)(9,31,11,29)(10,30,12,32)(13,21,15,23)(14,24,16,22)(17,27,19,25)(18,26,20,28)',
 '(1,14)(2,15)(3,16)(4,13)(5,22)(6,23)(7,24)(8,21)(9,17)(10,18)(11,19)(12,20)(25,29)(26,30)(27,31)(28,32)',
 '(1,20,12,14)(2,19,9,13)(3,18,10,16)(4,17,11,15)(5,24,30,28)(6,23,31,27)(7,22,32,26)(8,21,29,25)',
 '(1,12)(2,9)(3,10)(4,11)(5,30)(6,31)(7,32)(8,29)(13,19)(14,20)(15,17)(16,18)(21,25)(22,26)(23,27)(24,28)',
 '(1,12)(2,11)(3,10)(4,9)(5,7)(13,15)(17,19)(21,25)(22,28)(23,27)(24,26)(30,32)',
 '(1,26)(2,27)(3,28)(4,25)(5,14)(6,15)(7,16)(8,13)(9,23)(10,24)(11,21)(12,22)(17,31)(18,32)(19,29)(20,30)

In [4]:
import random
from base64 import b64encode, b64decode
from functools import reduce

SEED = 0xc0ffee0f

G = PermutationGroup([
    [(1,2,3,4),(5,6,7,8),(9,10,11,12),(13,14,15,16),(17,18,19,20),(21,22,23,24),(25,26,27,28),(29,30,31,32)],
    [(1,24,3,22),(2,23,4,21),(5,14,7,16),(6,13,8,15),(9,27,11,25),(10,26,12,28),(17,31,19,29),(18,30,20,32)],
    [(1,5,12,30),(2,6,9,31),(3,7,10,32),(4,8,11,29),(13,25,19,21),(14,26,20,22),(15,27,17,23),(16,28,18,24)],
    [(1,26),(2,27),(3,28),(4,25),(5,14),(6,15),(7,16),(8,13),(9,23),(10,24),(11,21),(12,22),(17,31),(18,32),(19,29),(20,30)]
])
num2e = [*G]
e2num = {y:x for x,y in enumerate(num2e)}
b64encode_map = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghiklmnopqrstuvwxyz0987654321+/="
b64decode_map = {c:i for i,c in enumerate(b64encode_map)}

def gen_random_mat(seed: int, size: int):
    random.seed(seed)
    return matrix(size, size, random.choices([0,1], k=size*size))

def mat_vec_mul(mat, vec):
    return [reduce(lambda x,y: x*y, (a^b for a,b in zip(vec, m))) for m in mat]

def hash_msg(msg: bytes):
    emsg = b64encode(msg).decode()
    sz = len(emsg)
    vec = [num2e[b64decode_map[c]] for c in emsg]
    mat = gen_random_mat(SEED, sz)
    for _ in range(5):
        # Since G is non-abelian so this is a pretty good hash function!
        vec = mat_vec_mul(mat, vec)
    return ''.join((b64encode_map[e2num[c]] for c in vec))

if __name__ == "__main__":
    from flag import flag # secret!!
    import re

    assert re.match(r"^grey\{.+\}$", flag.decode())
    ct = hash_msg(flag)
    print(ct)

# Program output:
# o4+OvfN1MlIwLNLcNv9EYzCbExIFNOhBIsCdH/=L6TdkieLfvZz=MPtUINP4TRv/MOlz


o4+OvfN1MlIwLNLcNv9EYzCbExIFNOhBIsCdH/=L6TdkieLfvZz=MPtUINP4TRv/MOlz


In [10]:
from itertools import product

e_ord = {}
for o,e in ((e.order(), e) for e in num2e):
    if o not in e_ord: e_ord[o] = []
    e_ord[o].append(e)

for a,b,c,d in product(e_ord[4], e_ord[4], e_ord[2], e_ord[2]):
    if a == b or c == d: continue
    if len(set(a^x * b^y * c^z * d^w for x,y,z,w in product(*map(range, (4,4,2,2))))) != 64: continue
    if b*c != c*b^(-1): continue
    if a*b*a^(-1) != b: continue
    if a*c*a^(-1) != c: continue
    break

e2tuple = {a^x * b^y * c^z * d^w: (x,y,z,w) for x,y,z,w in product(*map(range, (4,4,2,2)))}
tuple2e = {y:x for x,y in e2tuple.items()}

In [11]:
def cast_vec(vec, dim):
    return [e2tuple[c][dim] for c in vec]

def my_hash(vec):
    sz = len(vec)
    mat = gen_random_mat(SEED, sz)
    for _ in range(5):
        vec = mat_vec_mul(mat, vec)
    return vec

def modq_kernel(matx, q):
    # Returns the mod_q (right) kernel of a given matrix, in the case
    # that q is not prime
    # Uses https://ask.sagemath.org/question/33890/how-to-find-kernel-of-a-matrix-in-mathbbzn/
    # Iterates over whole kernel to find a matrix that generates it, likely slow
    source_dim = matx.ncols()
    target_dim = matx.nrows()
    domain = ZZ^source_dim / (q * ZZ^source_dim)
    codomain = ZZ^target_dim / (q * ZZ^target_dim)
    phi = domain.hom([codomain(matx.columns()[i]) for i in range(source_dim)])
    
    full_kernel = matrix([domain(b) for b in phi.kernel()]).T

    pivot_cols = full_kernel.echelon_form().pivots()
    reduced_kernel = matrix(full_kernel.columns()[i] for i in pivot_cols)
    return reduced_kernel

In [5]:
import sys
sys.path.append("dist")

from base64 import b64encode, b64decode
import re, random
from sage.modules.finite_submodule_iter import FiniteZZsubmodule_iterator
from itertools import product
from functools import reduce

##########################################
########## START From challenge ##########
##########################################

SEED = 0xc0ffee0f

# https://people.maths.bris.ac.uk/~matyd/GroupNames/61/Q8s5D4.html
G = PermutationGroup([
    [(1,2,3,4),(5,6,7,8),(9,10,11,12),(13,14,15,16),(17,18,19,20),(21,22,23,24),(25,26,27,28),(29,30,31,32)],
    [(1,24,3,22),(2,23,4,21),(5,14,7,16),(6,13,8,15),(9,27,11,25),(10,26,12,28),(17,31,19,29),(18,30,20,32)],
    [(1,5,12,30),(2,6,9,31),(3,7,10,32),(4,8,11,29),(13,25,19,21),(14,26,20,22),(15,27,17,23),(16,28,18,24)],
    [(1,26),(2,27),(3,28),(4,25),(5,14),(6,15),(7,16),(8,13),(9,23),(10,24),(11,21),(12,22),(17,31),(18,32),(19,29),(20,30)]
])
num2e = [*G]
e2num = {y:x for x,y in enumerate(num2e)}
b64encode_map = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghiklmnopqrstuvwxyz0987654321+/="
b64decode_map = {c:i for i,c in enumerate(b64encode_map)}

def gen_random_mat(seed: int, size: int):
    random.seed(seed)
    return matrix(size, size, random.choices([0,1], k=size*size))

def mat_vec_mul(mat, vec):
    return [reduce(lambda x,y: x*y, (a^b for a,b in zip(vec, m))) for m in mat]

def hash_msg(msg: bytes):
    emsg = b64encode(msg).decode()
    sz = len(emsg)
    vec = [num2e[b64decode_map[c]] for c in emsg]
    mat = gen_random_mat(SEED, sz)
    for _ in range(5):
        # Since G is non-abelian so this is a pretty good hash function!
        vec = mat_vec_mul(mat, vec)
    return ''.join((b64encode_map[e2num[c]] for c in vec))

ct = "o4+OvfN1MlIwLNLcNv9EYzCbExIFNOhBIsCdH/=L6TdkieLfvZz=MPtUINP4TRv/MOlz"

########################################
########## END From challenge ##########
########################################

def find_generators():
    """
    Find elements a,b,c,d of G such that
    1. All elements of G are uniquely of the form: a^x b^y c^z d^w
    2. a^x b^y c^z d^w |-> ((a, (b, c)), d) exibits an isomorphism
       G -> (C4 × (C4 ⋊ C2)) ⋊ C2
             │     │     │      │
            0th   1st   2nd    3rd - component
    """

    e_ord = {}
    for o,e in ((e.order(), e) for e in num2e):
        if o not in e_ord: e_ord[o] = []
        e_ord[o].append(e)

    for a,b,c,d in product(e_ord[4], e_ord[4], e_ord[2], e_ord[2]):
        if a == b or c == d: continue
        if len(set(a^x * b^y * c^z * d^w for x,y,z,w in product(*map(range, (4,4,2,2))))) != 64: continue
        if b*c != c*b^(-1): continue
        if a*b*a^(-1) != b: continue
        if a*c*a^(-1) != c: continue
        break
    
    return a,b,c,d

a,b,c,d = find_generators()
# Maps elements in G to-and-fro tuple representation. See `find_generators`.
e2tuple = {a^x * b^y * c^z * d^w: (x,y,z,w) for x,y,z,w in product(*map(range, (4,4,2,2)))}
tuple2e = {y:x for x,y in e2tuple.items()}

def cast_vec(vec, dim):
    """
    Cast a vector of elements in G to the `dim`-th
    component. See `find_generators`
    """
    return [e2tuple[c][dim] for c in vec]

def my_hash(vec):
    """
    Like `hash_msg but without the base64 part`
    """
    sz = len(vec)
    mat = gen_random_mat(SEED, sz)
    for _ in range(5):
        vec = mat_vec_mul(mat, vec)
    return vec

def modq_kernel(matx, q):
    """
    Solves for the kernel of `matx` which is a matrix
    with elements in ZZ/`q`ZZ
    """
    # From: https://ask.sagemath.org/question/49365/computing-the-mod-q-kernel-of-a-matrix-for-q-composite/
    source_dim = matx.ncols()
    target_dim = matx.nrows()
    domain = ZZ^source_dim / (q * ZZ^source_dim)
    codomain = ZZ^target_dim / (q * ZZ^target_dim)
    phi = domain.hom([codomain(matx.columns()[i]) for i in range(source_dim)])
    
    full_kernel = matrix([domain(b) for b in phi.kernel()]).T

    pivot_cols = full_kernel.echelon_form().pivots()
    reduced_kernel = matrix(full_kernel.columns()[i] for i in pivot_cols)
    return reduced_kernel.change_ring(Zmod(q))

def matq_solve_right_all(mat, vec, q):
    """
    Solves for all vectors u such that
    `mat * u = vec`, where we are working
    in ZZ/`q`ZZ.
    """
    v = mat.solve_right(vec)
    if is_prime(q):
        return [v + k for k in mat.right_kernel()]
    kernel = modq_kernel(mat, q)
    if kernel.nrows() > 8: raise Exception("Too many solutions, regen challenge")
    return [v + k for k in FiniteZZsubmodule_iterator([*kernel])]


sz = len(ct)
ct_vec = [num2e[b64decode_map[c]] for c in ct]
possible = []

# By working in the quotient G -> G/(C4 × (C4 ⋊ C2)),
# isomorphic to C2,
# find all possible 3rd component of the flag.

ct_vec_3 = vector(Zmod(2), cast_vec(ct_vec, 3))
mat_3 = gen_random_mat(SEED, sz).change_ring(Zmod(2))^5
pos_pt_3s = matq_solve_right_all(mat_3, ct_vec_3, 2)

print("No. possible 3rd component:", len(pos_pt_3s), end="\n\n")
for pt_vec_3 in pos_pt_3s:
    
    # By commutating with (known) 3rd component,
    # and working in the quotient
    # (C4 × (C4 ⋊ C2)) -> (C4 × (C4 ⋊ C2))/C4/C4,
    # isomorphic to C2,
    # find all possible 2nd component of the flag.

    mat = []
    for j in range(sz):
        v = [tuple2e[(0,0,int(i==j),b)] for i,b in enumerate(pt_vec_3)]
        v = cast_vec(my_hash(v), 2)
        mat.append(v)
    mat = matrix(Zmod(2), mat).T

    known = my_hash([tuple2e[(0,0,0,b)] for b in pt_vec_3])
    diff = [a * b^(-1) for a,b in zip(ct_vec, known)]
    diff = vector(Zmod(2), cast_vec(diff, 2))
    pos_pt_2s = matq_solve_right_all(mat, diff, 2)

    print(f"Assuming 3rd component is: {''.join(map(str, pt_vec_3))},\n"
           "No. possible 2nd component:", len(pos_pt_2s), end="\n\n")
    for pt_vec_2 in pos_pt_2s:
        
        # By commutating with (known) 3rd component,
        # and working in the quotient
        # (C4 × (C4 ⋊ C2)) -> (C4 × (C4 ⋊ C2))/C4,
        # and commutating with (known) 2nd component,
        # we can work in C4 to
        # find all possible 1st component of the flag.

        mat = []
        for j in range(sz):
            v = [tuple2e[(0, int(i==j), *v)] 
                 for i, v
                 in enumerate(zip(pt_vec_2, pt_vec_3))]
            v = cast_vec(my_hash(v), 1)
            mat.append(v)
        mat = matrix(Zmod(4), mat).T

        known = my_hash([tuple2e[(0, 0, *v)] for v in zip(pt_vec_2, pt_vec_3)])
        diff = [a * b^(-1) for a,b in zip(ct_vec, known)]
        diff = vector(Zmod(4), cast_vec(diff, 1))
        pos_pt_1s = matq_solve_right_all(mat, diff, 4)
        
        # Since `matq_solve_right_all` is super slow when
        # the modulus isn't prime, we can optimize by filtering out
        # possible 1st component of the flag since we know
        # the flag has to start with `grey{`.
        
        knownflag_b64 = b64encode(b"grey{")[:6].decode()
        possible_pt_vec_1_values_first_6 = [
            set((tuple2e[(x,0,0,0)]
                * num2e[b64decode_map[c]]
                * tuple2e[(0,0,0,b3)] 
                * tuple2e[(0,0,b2,0)] for x in range(4))
               ) 
            - set((tuple2e[(0,x,0,0)] for x in range(4)))
            for c,b2,b3
            in zip(knownflag_b64, pt_vec_2, pt_vec_3)]
        possible_pt_vec_1_values_first_6 = [
            set([e2tuple[x][1] for x in c]) 
            for c in possible_pt_vec_1_values_first_6]

        pos_pt_1s = filter(
            lambda s: all(a in b for a,b in zip(s, possible_pt_vec_1_values_first_6)),
            pos_pt_1s)
        pos_pt_1s = list(pos_pt_1s)

        print(f"Assuming 3rd component is: {''.join(map(str, pt_vec_3))},\n"
              f"Assuming 2nd component is: {''.join(map(str, pt_vec_2))},\n"
               "No. possible 1st component:", len(pos_pt_1s), end="\n\n")
        for pt_vec_1 in pos_pt_1s:
            
            # By commutating with (known) 3rd component,
            # and working in the quotient
            # (C4 × (C4 ⋊ C2)) -> (C4 × (C4 ⋊ C2))/(C4 ⋊ C2),
            # isomorphic to C4,
            # find all possible 1st component of the flag.

            mat = []
            for j in range(sz):
                v = [tuple2e[(int(i==j), *v)] 
                    for i, v
                    in enumerate(zip(pt_vec_1, pt_vec_2, pt_vec_3))]
                v = cast_vec(my_hash(v), 0)
                mat.append(v)
            mat = matrix(Zmod(4), mat).T

            known1 = my_hash([tuple2e[(0,0,0,b3)] for b3 in pt_vec_3])
            known2 = my_hash([tuple2e[(0,*v,0)] for v in zip(pt_vec_1, pt_vec_2)])
            diff = [a * b^(-1) * c^(-1) for a,b,c in zip(ct_vec, known1, known2)]
            diff = vector(Zmod(4), cast_vec(diff, 0))
            pos_pt_0s = matq_solve_right_all(mat, diff, 4)
            
            print(f"Assuming 3rd component is: {''.join(map(str, pt_vec_3))},\n"
                  f"Assuming 2nd component is: {''.join(map(str, pt_vec_2))},\n"
                  f"Assuming 1st component is: {''.join(map(str, pt_vec_1))},\n"
                   "No. possible 0th component:", len(pos_pt_0s), end="\n\n")
            for pt_vec_0 in pos_pt_0s:
                
                # Finally, with all 4 components,
                # we can check if the components we have found
                # is actually the flag.

                b64_flag = ''.join(
                    b64encode_map[e2num[a^x * b^y * c^z * d^w]]
                    for x,y,z,w in zip(pt_vec_0, pt_vec_1, pt_vec_2, pt_vec_3)
                )
                try: decoded = b64decode(b64_flag).decode()
                except: continue
                if not re.match(r"^grey\{.+\}$", decoded): continue
                if decoded in possible: continue

                possible.append(decoded)
                print("Possible flag:", decoded)

# No. possible 3rd component: 2
# 
# Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
# No. possible 2nd component: 2
# 
# Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
# Assuming 2nd component is: 11011100001011010000110101101010101101111100010000001011000110100111,
# No. possible 1st component: 8
# 
# Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
# Assuming 2nd component is: 11011100001011010000110101101010101101111100010000001011000110100111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
# Assuming 2nd component is: 11011100001011010000110101101010101101111100010000001011000110100111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
# Assuming 2nd component is: 11011100001011010000110101101010101101111100010000001011000110100111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
# Assuming 2nd component is: 11011100001011010000110101101010101101111100010000001011000110100111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
# Assuming 2nd component is: 11011100001011010000110101101010101101111100010000001011000110100111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
# Assuming 2nd component is: 11011100001011010000110101101010101101111100010000001011000110100111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
# Assuming 2nd component is: 11011100001011010000110101101010101101111100010000001011000110100111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
# Assuming 2nd component is: 11011100001011010000110101101010101101111100010000001011000110100111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
# Assuming 2nd component is: 11001100111111101011111101111110111011011110000111010101010010110111,
# No. possible 1st component: 0
# 
# Assuming 3rd component is: 10011110100110111010110001101011101011001100000111000000010110110000,
# No. possible 2nd component: 2
# 
# Assuming 3rd component is: 10011110100110111010110001101011101011001100000111000000010110110000,
# Assuming 2nd component is: 11011100001011010000110101101010101101111100010000001011000110100111,
# No. possible 1st component: 0
# 
# Assuming 3rd component is: 10011110100110111010110001101011101011001100000111000000010110110000,
# Assuming 2nd component is: 11001100111111101011111101111110111011011110000111010101010010110111,
# No. possible 1st component: 8
# 
# Assuming 3rd component is: 10011110100110111010110001101011101011001100000111000000010110110000,
# Assuming 2nd component is: 11001100111111101011111101111110111011011110000111010101010010110111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Possible flag: grey{OwO_we-h4ve-4-grOUp_tHe0ry_M4STER1337!!-HER3}
# Assuming 3rd component is: 10011110100110111010110001101011101011001100000111000000010110110000,
# Assuming 2nd component is: 11001100111111101011111101111110111011011110000111010101010010110111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Assuming 3rd component is: 10011110100110111010110001101011101011001100000111000000010110110000,
# Assuming 2nd component is: 11001100111111101011111101111110111011011110000111010101010010110111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Assuming 3rd component is: 10011110100110111010110001101011101011001100000111000000010110110000,
# Assuming 2nd component is: 11001100111111101011111101111110111011011110000111010101010010110111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Assuming 3rd component is: 10011110100110111010110001101011101011001100000111000000010110110000,
# Assuming 2nd component is: 11001100111111101011111101111110111011011110000111010101010010110111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Assuming 3rd component is: 10011110100110111010110001101011101011001100000111000000010110110000,
# Assuming 2nd component is: 11001100111111101011111101111110111011011110000111010101010010110111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Assuming 3rd component is: 10011110100110111010110001101011101011001100000111000000010110110000,
# Assuming 2nd component is: 11001100111111101011111101111110111011011110000111010101010010110111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32
# 
# Assuming 3rd component is: 10011110100110111010110001101011101011001100000111000000010110110000,
# Assuming 2nd component is: 11001100111111101011111101111110111011011110000111010101010010110111,
# Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
# No. possible 0th component: 32

No. possible 3rd component: 2

Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
No. possible 2nd component: 2

Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
Assuming 2nd component is: 11011100001011010000110101101010101101111100010000001011000110100111,
No. possible 1st component: 8

Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
Assuming 2nd component is: 11011100001011010000110101101010101101111100010000001011000110100111,
Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
No. possible 0th component: 32

Assuming 3rd component is: 10001110010010000001111001111111111101101110010000011110000010100000,
Assuming 2nd component is: 11011100001011010000110101101010101101111100010000001011000110100111,
Assuming 1st component is: 13123213302333211003100012300121230110003223323202103033102102203301,
No

In [2]:
is_prime(2)

True

In [4]:
pt_vec = [Element(Element.b64decode_map[c]) for c in b64encode(flag).decode()]
pt_vec_3 = vector(GF(2), cast_vec(pt_vec, 3))
pt_vec_2 = vector(GF(2), cast_vec(pt_vec, 2))
pt_vec_1 = vector(Zmod(4), cast_vec(pt_vec, 1))
pt_vec_0 = vector(Zmod(4), cast_vec(pt_vec, 0))
pt_vec_0

(0, 1, 2, 0, 3, 1, 2, 3, 0, 1, 0, 1, 0, 3, 2, 2, 3, 3, 0, 3, 0, 3, 1, 1, 3, 3, 0, 2, 1, 2, 0, 3, 1, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 3, 0, 1, 0, 3, 0, 2, 3, 0, 0, 3, 0, 1, 1, 3)

In [165]:
#ct = "WuDmWLTVJsmW0eqf/JDpipuFbv2=YyQu3lmbniBEgTYtE6CahImeZMsGKB6u8YvHL1gu"
sz = len(ct)
ct_vec = [Element(Element.b64decode_map[c]) for c in ct]

ct_vec_3 = vector(Zmod(2), cast_vec(ct_vec, 3))
mat_3 = gen_random_mat(SEED, sz).change_ring(Zmod(2))^5
# Recover possible pt_vec_3
_tmp = mat_3.solve_right(ct_vec_3)
rec_pt_3s = {tuple(_tmp + k): [] for k in mat_3.right_kernel()}

print(len(rec_pt_3s))
assert pt_vec_3 in [vector(Zmod(2), v) for v in rec_pt_3s.keys()]

mat_2 = matrix(Zmod(2), [
    cast_vec(my_hash([tuple2e[(0,0,int(i==j),b)] for i,b in enumerate(pt_vec_3)]), 2)
    for j in range(sz)
]).T

known = my_hash([tuple2e[(0,0,0,b)] for b in pt_vec_3])
diff_2 = vector(Zmod(2), cast_vec([a + b*(-1) for a,b in zip(ct_vec, known)], 2))
_tmp = mat_2.solve_right(diff_2)
rec_pt_2s = {tuple(_tmp + k): [] for k in mat_2.right_kernel()}

print(len(rec_pt_2s))
assert pt_vec_2 in [vector(Zmod(2), v) for v in rec_pt_2s.keys()]

mat_1 = matrix(Zmod(4), [
    cast_vec(my_hash([tuple2e[(0,int(i==j),b2,b3)] for i,(b2,b3) in enumerate(zip(pt_vec_2, pt_vec_3))]), 1)
    for j in range(sz)
]).T

known = my_hash([tuple2e[(0,0,b2,b3)] for b2,b3 in zip(pt_vec_2, pt_vec_3)])
diff_1 = vector(Zmod(4), cast_vec([a + b*(-1) for a,b in zip(ct_vec, known)], 1))
_tmp = mat_1.solve_right(diff_1)
kernel = modq_kernel(mat_1, 4)
if kernel.nrows() > 5: raise Exception("Too many solutions, regen challenge")
if kernel.nrows() == 0: rec_pt_1s = [_tmp]
else: rec_pt_1s = [_tmp + k for k in FiniteZZsubmodule_iterator([*kernel.change_ring(Zmod(4))])]

print(len(rec_pt_1s))
assert pt_vec_1 in rec_pt_1s

mat_0 = matrix(Zmod(4), [
    cast_vec(my_hash([tuple2e[(int(i==j),b1,b2,b3)] for i,(b1,b2,b3) in enumerate(zip(pt_vec_1, pt_vec_2, pt_vec_3))]), 0)
    for j in range(sz)
]).T

known1 = my_hash([tuple2e[(0,0,0,b3)] for b1,b2,b3 in zip(pt_vec_1, pt_vec_2, pt_vec_3)])
known2 = my_hash([tuple2e[(0,b1,b2,0)] for b1,b2,b3 in zip(pt_vec_1, pt_vec_2, pt_vec_3)])
diff_0 = vector(Zmod(4), cast_vec([a+b*(-1)+c*(-1) for a,b,c in zip(ct_vec, known1, known2)], 0))
_tmp = mat_0.solve_right(diff_0)
if kernel.nrows() > 5: raise Exception("Too many solutions, regen challenge")
if kernel.nrows() == 0: rec_pt_0s = [_tmp]
else: rec_pt_0s = [_tmp + k for k in FiniteZZsubmodule_iterator([*kernel.change_ring(Zmod(4))])]
    
print(len(rec_pt_0s))
assert pt_vec_0 in rec_pt_0s

2
2
32
32


In [ ]:
b64_flag = ''.join((
    Element.b64encode_map[Element.e2num[a^int(x) * b^int(y) * c^int(z) * d^int(w)]]
    for x,y,z,w in zip(pt_vec_0, pt_vec_1, pt_vec_2, pt_vec_3)
))
print(b64_flag)
base64.b64decode(b64_flag)

In [60]:
pt_vec_0

(2, 3, 2, 2, 3, 1, 2, 3, 2, 1, 2, 1, 2, 3, 0, 2, 3, 1, 2, 1, 2, 1, 1, 3, 1, 1, 0, 0, 1, 2, 2, 3, 1, 3, 2, 0, 0, 1, 2, 0, 1, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 3, 2, 1, 2, 1, 2, 0, 3, 2, 0, 1, 2, 1, 1, 3)

In [7]:
known = my_hash([tuple2e[(0,0,b2,b3)] for b2,b3 in zip(pt_vec_2, pt_vec_3)])
diff_1 = vector(Zmod(4), cast_vec([a + b*(-1) for a,b in zip(ct_vec, known)], 1))

In [11]:
_tmp, pt_vec_1

((1, 3, 1, 2, 3, 2, 1, 3, 3, 0, 2, 3, 3, 3, 2, 1, 1, 0, 0, 3, 1, 0, 0, 0, 1, 2, 3, 0, 0, 1, 2, 1, 2, 3, 0, 1, 1, 0, 0, 0, 3, 2, 2, 3, 3, 2, 3, 2, 0, 2, 1, 0, 3, 0, 3, 3, 1, 0, 2, 1, 0, 2, 2, 0, 3, 3, 0, 1),
 (1, 3, 1, 2, 3, 2, 1, 3, 3, 0, 2, 3, 3, 3, 2, 1, 1, 0, 0, 3, 1, 0, 0, 0, 1, 2, 3, 0, 0, 1, 2, 1, 2, 3, 0, 1, 1, 0, 0, 0, 3, 2, 2, 3, 3, 2, 3, 2, 0, 2, 1, 0, 3, 0, 3, 3, 1, 0, 2, 1, 0, 2, 2, 0, 3, 3, 0, 1))

In [67]:
mat_0 = matrix(Zmod(4), [
    cast_vec(my_hash([tuple2e[(int(i==j),b1,b2,b3)] for i,(b1,b2,b3) in enumerate(zip(pt_vec_1, pt_vec_2, pt_vec_3))]), 0)
    for j in range(sz)
]).T

known1 = my_hash([tuple2e[(0,0,0,b3)] for b1,b2,b3 in zip(pt_vec_1, pt_vec_2, pt_vec_3)])
known2 = my_hash([tuple2e[(0,b1,b2,0)] for b1,b2,b3 in zip(pt_vec_1, pt_vec_2, pt_vec_3)])
diff_0 = vector(Zmod(4), cast_vec([a+b*(-1)+c*(-1) for a,b,c in zip(ct_vec, known1, known2)], 0))
_tmp = mat_0.solve_right(diff_0)
rec_pt_0s = {tuple(_tmp + k): [] for k in modq_kernel(mat_0, 4)}

print(len(rec_pt_0s))
assert pt_vec_0 in [vector(Zmod(4), v) for v in rec_pt_0s.keys()]

7


In [65]:
mat_0 * pt_vec_0

(0, 3, 0, 0, 0, 1, 0, 1, 2, 1, 2, 1, 3, 2, 2, 1, 1, 1, 3, 0, 3, 0, 2, 2, 0, 1, 0, 0, 3, 1, 0, 3, 3, 3, 2, 1, 2, 3, 2, 3, 2, 0, 0, 3, 0, 3, 0, 2, 2, 0, 1, 0, 3, 3, 3, 3, 1, 0, 0, 0, 1, 3, 0, 2, 3, 2, 1, 0)

In [66]:
diff_0

(0, 3, 0, 0, 0, 1, 0, 1, 2, 1, 2, 1, 3, 2, 2, 1, 1, 1, 3, 0, 3, 0, 2, 2, 0, 1, 0, 0, 3, 1, 0, 3, 3, 3, 2, 1, 2, 3, 2, 3, 2, 0, 0, 3, 0, 3, 0, 2, 2, 0, 1, 0, 3, 3, 3, 3, 1, 0, 0, 0, 1, 3, 0, 2, 3, 2, 1, 0)

In [121]:
import numpy as np
def kernel_mod4(mat):
    x = mat.change_ring(GF(2)).right_kernel()
    y = [*matrix(GF(2), np.array(mat, dtype=np.int32)//2).right_kernel()]
    aa = [vector(Zmod(4), [*v])*2 + w for v,w in product(y,x)]
    return [vector(Zmod(4), v) for v in eval(str(aa))]

In [58]:
mat_1 * pt_vec_1

(2, 2, 2, 0, 3, 3, 1, 0, 0, 1, 1, 3, 3, 3, 0, 2, 0, 2, 3, 2, 2, 2, 1, 2, 0, 1, 2, 2, 2, 3, 1, 1, 0, 2, 3, 1, 2, 0, 0, 1, 1, 0, 1, 3, 1, 0, 0, 3, 1, 2, 1, 2, 3, 2, 2, 1, 0, 1, 1, 0, 3, 3, 1, 3, 2, 2, 1, 0, 3, 1, 1, 3, 0, 2, 2, 1, 1, 2, 1, 0, 1, 3, 1, 1, 0, 2, 1, 0, 0, 2, 2, 3, 1, 0, 1, 1)

In [59]:
diff_1

(2, 2, 2, 0, 3, 3, 1, 0, 0, 1, 1, 3, 3, 3, 0, 2, 0, 2, 3, 2, 2, 2, 1, 2, 0, 1, 2, 2, 2, 3, 1, 1, 0, 2, 3, 1, 2, 0, 0, 1, 1, 0, 1, 3, 1, 0, 0, 3, 1, 2, 1, 2, 3, 2, 2, 1, 0, 1, 1, 0, 3, 3, 1, 3, 2, 2, 1, 0, 3, 1, 1, 3, 0, 2, 2, 1, 1, 2, 1, 0, 1, 3, 1, 1, 0, 2, 1, 0, 0, 2, 2, 3, 1, 0, 1, 1)

In [48]:
pt_vec_2 in [mat_2.solve_right(diff_2) + k for k in mat_2.right_kernel()]

True

In [137]:
pt_vec = [Element(Element.b64decode_map[c]) for c in b64encode(flag).decode()]
pt_vec_3 = vector(GF(2), cast_vec(pt_vec, 3))
pt_vec_2 = vector(GF(2), cast_vec(pt_vec, 2))
pt_vec_1 = vector(Zmod(4), cast_vec(pt_vec, 1))
pt_vec_0 = vector(Zmod(4), cast_vec(pt_vec, 0))
pt_vec_0

(0, 1, 2, 0, 3, 1, 2, 3, 0, 1, 0, 1, 0, 3, 2, 2, 3, 3, 0, 3, 0, 3, 1, 1, 3, 3, 0, 2, 1, 2, 0, 3, 1, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 3, 0, 1, 0, 3, 0, 2, 3, 0, 0, 3, 0, 1, 1, 3)